In [6]:
import os
import re
import string


In [3]:
LABEL_DRUG_PROTEIN = 'DRUG-PROTEIN'
LABEL_CHEMICAL = 'CHEMICAL'
LABEL_DISEASE = 'DISEASE'
LABEL_SPECIES = 'SPECIES'

LABEL_PATH = {
    LABEL_DRUG_PROTEIN: ['BC2GM', 'JNLPBA'],
    LABEL_CHEMICAL: ['BC4CHEMD','BC5CDR-chem'],
    LABEL_DISEASE: ['BC5CDR-disease', 'NCBI-disease'],    
    LABEL_SPECIES: ['linnaeus', 's800']
}

TRAIN_PATH = 'NER-Data'

FILE_TYPES = ["train.tsv", "devel.tsv", "test.tsv"]



In [13]:
dataset_ner_file = os.path.join(TRAIN_PATH, "BC2GM", FILE_TYPES[0])
with open(dataset_ner_file) as f_ner: 
    sentenca = ""
    entities = []
    lista_sentenca = []
    ini_entity_atual = 0
    pos_atual = 0
    entidade_atual = ""
    
    for linha in f_ner:
        if linha != "\n":
            if ("\tO" in linha):
                if len(entidade_atual) > 0:
                    entities.append((entidade_atual[0:-1], ini_entity_atual, pos_atual-1))
                    entidade_atual = ""
                    ini_entity_atual = 0
                linha_tratada = linha.replace("\tO","").replace("\n", " ")
                
            elif("\tB" in linha):
                if len(entidade_atual) > 0:
                    entities.append((entidade_atual[0:-1], ini_entity_atual, len(entidade_atual) -1))                    
                ini_entity_atual = pos_atual
                linha_tratada = linha.replace("\tB","").replace("\n", " ")
                entidade_atual = linha_tratada                
            elif("\tI" in linha):
                linha_tratada = linha.replace("\tI","").replace("\n", " ")
                if linha_tratada[0] in string.punctuation:
                    linha_tratada = linha_tratada[0:-1]
                    pos_atual = pos_atual - 1
                    entidade_atual = entidade_atual[0:-1] + linha_tratada
                    sentenca = sentenca[0:-1]                    
                else:
                    entidade_atual += linha_tratada
                    
            pos_atual += len(linha_tratada)
            sentenca = sentenca + linha_tratada
        elif linha == "\n":
            lista_sentenca.append((sentenca, entities))
            sentenca = ""
        if len(lista_sentenca) > 2:
            break
        
    

In [14]:
lista_sentenca

[('Immunohistochemical staining was positive for S-100 in all 9 cases stained , positive for HMB-45 in 9 ( 90 % ) of 10 , and negative for cytokeratin in all 9 cases in which myxoid melanoma remained in the block after previous sections . ',
  [('S-100', 46, 51),
   ('HMB-45', 90, 96),
   ('cytokeratin', 136, 147),
   ('Chloramphenicol acetyltransferase', 236, 269),
   ('IE86', 302, 306),
   ('IE promoter', 347, 358),
   ('HCMV early promoter', 375, 394),
   ('IE86 protein', 473, 485),
   ('DNA repair gene', 494, 509),
   ('RecA', 558, 562)]),
 ('Chloramphenicol acetyltransferase assays examining the ability of IE86 to repress activity from the HCMV major IE promoter or activate the HCMV early promoter for the 2 . 2 - kb class of RNAs demonstrated the functional integrity of the IE86 protein . ',
  [('S-100', 46, 51),
   ('HMB-45', 90, 96),
   ('cytokeratin', 136, 147),
   ('Chloramphenicol acetyltransferase', 236, 269),
   ('IE86', 302, 306),
   ('IE promoter', 347, 358),
   ('HCMV ea